In [1]:
import pandas as pd
import numpy as np
import os
import json
from shapely.geometry import Polygon
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as Polygon_patch
from matplotlib.patches import Rectangle as Rectangle_patch

In [2]:
yolo_part_mapping = {0: 'Traffic-sings',
 1: 'forb_ahead',
 2: 'forb_left',
 3: 'forb_overtake',
 4: 'forb_right',
 5: 'forb_speed_over_10',
 6: 'forb_speed_over_100',
 7: 'forb_speed_over_130',
 8: 'forb_speed_over_20',
 9: 'forb_speed_over_30',
 10: 'forb_speed_over_40',
 11: 'forb_speed_over_5',
 12: 'forb_speed_over_50',
 13: 'forb_speed_over_60',
 14: 'forb_speed_over_70',
 15: 'forb_speed_over_80',
 16: 'forb_speed_over_90',
 17: 'forb_stopping',
 18: 'forb_trucks',
 19: 'forb_u_turn',
 20: 'forb_weight_over_3.5t',
 21: 'forb_weight_over_7.5t',
 22: 'info_bus_station',
 23: 'info_crosswalk',
 24: 'info_highway',
 25: 'info_one_way_traffic',
 26: 'info_parking',
 27: 'info_taxi_parking',
 28: 'mand_bike_lane',
 29: 'mand_left',
 30: 'mand_left_right',
 31: 'mand_pass_left',
 32: 'mand_pass_left_right',
 33: 'mand_pass_right',
 34: 'mand_right',
 35: 'mand_roundabout',
 36: 'mand_straigh_left',
 37: 'mand_straight',
 38: 'mand_straight_right',
 39: 'prio_give_way',
 40: 'prio_priority_road',
 41: 'prio_stop',
 42: 'warn_children',
 43: 'warn_construction',
 44: 'warn_crosswalk',
 45: 'warn_cyclists',
 46: 'warn_domestic_animals',
 47: 'warn_other_dangers',
 48: 'warn_poor_road_surface',
 49: 'warn_roundabout',
 50: 'warn_slippery_road',
 51: 'warn_speed_bumper',
 52: 'warn_traffic_light',
 53: 'warn_tram',
 54: 'warn_two_way_traffic',
 55: 'warn_wild_animals'}

In [ ]:
import os
import pandas as pd
from collections import defaultdict

def bbox_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calculate the coordinates of the intersection rectangle
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    # Calculate the area of intersection rectangle
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    # Calculate the area of both bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    # Calculate the IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    
    return iou

def calculate_metrics(gt_dir, pred_dir, iou_threshold=0.5):
    """
    Calculate performance metrics for object detection.
    """
    gt_boxes = defaultdict(lambda: defaultdict(list))  # {image_name: {class_id: [...]}}
    pred_boxes = defaultdict(lambda: defaultdict(list))  # {image_name: {class_id: [...]}}
    
    # Parse ground truth files
    for gt_file in os.listdir(gt_dir):
        image_name = os.path.splitext(gt_file)[0]
        with open(os.path.join(gt_dir, gt_file), 'r') as f:
            for line in f:
                class_id, x, y, w, h = map(float, line.split())
                gt_boxes[image_name][int(class_id)].append((x, y, w, h))
    
    # Parse predicted files
    for pred_file in os.listdir(pred_dir):
        image_name = os.path.splitext(pred_file)[0]
        with open(os.path.join(pred_dir, pred_file), 'r') as f:
            for line in f:
                parts = line.split()
                class_id, x, y, w, h, conf = map(float, parts[:6])
                pred_boxes[image_name][int(class_id)].append((x, y, w, h, conf))
    
    metrics = defaultdict(dict)
    image_metrics = defaultdict(dict)
    
    for image_name in set(gt_boxes.keys()) | set(pred_boxes.keys()):
        for class_id in set(gt_boxes[image_name].keys()) | set(pred_boxes[image_name].keys()):
            tp, fp, fn = 0, 0, 0
            gt_count = len(gt_boxes[image_name][class_id])
            pred_count = len(pred_boxes[image_name][class_id])
            
            # Sort predicted boxes by confidence
            sorted_pred_boxes = sorted(pred_boxes[image_name][class_id], key=lambda x: x[-1], reverse=True)
            
            # Calculate TP, FP, and FN
            for pred_box in sorted_pred_boxes:
                found = False
                for gt_box in gt_boxes[image_name][class_id]:
                    iou = bbox_iou(pred_box[:4], gt_box)
                    if iou >= iou_threshold:
                        tp += 1
                        gt_boxes[image_name][class_id].remove(gt_box)
                        found = True
                        break
                if not found:
                    fp += 1
            fn = gt_count - tp
            
            # Calculate precision, recall, and F-score
            if tp + fp > 0:
                precision = tp / (tp + fp)
            else:
                precision = 0
            if tp + fn > 0:
                recall = tp / (tp + fn)
            else:
                recall = 0
            if precision + recall > 0:
                f_score = 2 * (precision * recall) / (precision + recall)
            else:
                f_score = 0
            
            # Store metrics
            metrics[(image_name, class_id)] = {
                'TP': tp,
                'FP': fp,
                'FN': fn,
                'pred_count': pred_count,
                'gt_count': gt_count,
                'precision': precision,
                'recall': recall,
                'f_score': f_score
            }
            
            # Store image-level metrics
            image_metrics[image_name][class_id] = {
                'TP': tp,
                'FP': fp,
                'FN': fn
            }
    
    # Calculate average precision
    for image_name, class_id in metrics.keys():
        ap = 0
        precisions = []
        recalls = []
        for conf in sorted([box[-1] for box in pred_boxes[image_name][class_id]], reverse=True):
            tp, fp, fn = 0, 0, 0
            for pred_box in pred_boxes[image_name][class_id]:
                if pred_box[-1] >= conf:
                    found = False
                    for gt_box in gt_boxes[image_name][class_id]:
                        iou = bbox_iou(pred_box[:4], gt_box)
                        if iou >= iou_threshold:
                            tp += 1
                            gt_boxes[image_name][class_id].remove(gt_box)
                            found = True
                            break
                    if not found:
                        fp += 1
                else:
                    fn += len(gt_boxes[image_name][class_id])
                    break
            if tp + fp > 0:
                precision = tp / (tp + fp)
            else:
                precision = 0
            if tp + fn > 0:
                recall = tp / (tp + fn)
            else:
                recall = 0
            precisions.append(precision)
            recalls.append(recall)
            ap += precision * (recalls[-1] - recalls[-2]) if len(recalls) > 1 else precision * recalls[0]
        metrics[(image_name, class_id)]['ap'] = ap
    
    # Create a DataFrame with all metrics
    df = pd.DataFrame.from_dict(metrics, orient='index')
    
    return df, image_metrics

In [ ]:
gt_dir = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''
pred_dir = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
metrics_df, image_metrics = calculate_metrics(gt_dir, pred_dir)
metrics_df

TP  FP  FN  pred_count  \
SNAG-00138_png_jpg.rf.e4ce2fca462fa470b496a1c1c... 34   1   0   0           1   
                                                   39   1   0   0           1   
SNAGDEEA-0106_png_jpg.rf.fe64282b23f1b59e9e2837... 1    1   0   0           1   
SNAG12-RADU_882_png.rf.9beb8b123d09ebeeef3d59ca... 34   0   0   1           0   
                                                   39   3   0   0           3   
...                                                    ..  ..  ..         ...   
SNAG12-RADU_914_png.rf.328ec5f103578ce447e0639c... 44   0   1   0           1   
SNAGDEEA-0117_png_jpg.rf.7d08df4979b9b8e95a6ae4... 34   1   0   0           1   
                                                   39   1   0   0           1   
SNAGDEEA-0190_png_jpg.rf.78b64407bbead4c186461e... 40   1   0   0           1   
SNAG-00162_png_jpg.rf.4de82d0a11a0c04b21e0a6e8f... 40   1   0   0           1   

                                                       gt_count  precision  \
SNAG-00138_png_jpg.rf.e4ce2fca462fa470b496a1c1c... 34         1        1.0   
                                                   39         1        1.0   
SNAGDEEA-0106_png_jpg.rf.fe64282b23f1b59e9e2837... 1          1        1.0   
SNAG12-RADU_882_png.rf.9beb8b123d09ebeeef3d59ca... 34         1        0.0   
                                                   39         3        1.0   
...                                                         ...        ...   
SNAG12-RADU_914_png.rf.328ec5f103578ce447e0639c... 44         0        0.0   
SNAGDEEA-0117_png_jpg.rf.7d08df4979b9b8e95a6ae4... 34         1        1.0   
                                                   39         1        1.0   
SNAGDEEA-0190_png_jpg.rf.78b64407bbead4c186461e... 40         1        1.0   
SNAG-00162_png_jpg.rf.4de82d0a11a0c04b21e0a6e8f... 40         1        1.0   

                                                       recall  f_score   ap  
SNAG-00138_png_jpg.rf.e4ce2fca462fa470b496a1c1c... 34     1.0      1.0  0.0  
                                                   39     1.0      1.0  0.0  
SNAGDEEA-0106_png_jpg.rf.fe64282b23f1b59e9e2837... 1      1.0      1.0  0.0  
SNAG12-RADU_882_png.rf.9beb8b123d09ebeeef3d59ca... 34     0.0      0.0  0.0  
                                                   39     1.0      1.0  0.0  
...                                                       ...      ...  ...  
SNAG12-RADU_914_png.rf.328ec5f103578ce447e0639c... 44     0.0      0.0  0.0  
SNAGDEEA-0117_png_jpg.rf.7d08df4979b9b8e95a6ae4... 34     1.0      1.0  0.0  
                                                   39     1.0      1.0  0.0  
SNAGDEEA-0190_png_jpg.rf.78b64407bbead4c186461e... 40     1.0      1.0  0.0  
SNAG-00162_png_jpg.rf.4de82d0a11a0c04b21e0a6e8f... 40     1.0      1.0  0.0  

[951 rows x 9 columns]

In [ ]:
metrics_df = metrics_df.reset_index()
metrics_df[metrics_df.level_0 == 'SNAG-00138_png_jpg.rf.e4ce2fca462fa470b496a1c1c896602b']

,level_0,level_1,TP,FP,FN,pred_count,gt_count,precision,recall,f_score,ap
0,SNAG-00138_png_jpg.rf.e4ce2fca462fa470b496a1c1...,34,1,0,0,1,1,1.0,1.0,1.0,0.0
1,SNAG-00138_png_jpg.rf.e4ce2fca462fa470b496a1c1...,39,1,0,0,1,1,1.0,1.0,1.0,0.0


In [ ]:
for image_name, class_metrics in image_metrics.items():
    print(f"Image: {image_name}")
    for class_id, metrics in class_metrics.items():
        print(f"Class {class_id}: TP={metrics['TP']}, FP={metrics['FP']}, FN={metrics['FN']}")

Image: SNAG-00034_png_jpg.rf.4180c2d18b3ea4b8e86ec849a5b9b506
Class 34: TP=1, FP=0, FN=0
Class 23: TP=1, FP=0, FN=0
Class 41: TP=1, FP=0, FN=0
Image: SNAGDEEA-0053_png_jpg.rf.82daa2ed03f0e2a921f4bbe96bbf561e
Class 53: TP=1, FP=0, FN=0
Class 23: TP=1, FP=0, FN=0
Class 41: TP=1, FP=0, FN=0
Image: SNAGDEEA-0121_png_jpg.rf.3154705f461537533c8e8a796fcfe950
Class 41: TP=1, FP=0, FN=0
Image: SNAGDEEA-0131_png_jpg.rf.af2a3f3f4f93d6ba52dcc1fefcd17674
Class 40: TP=1, FP=0, FN=0
Class 51: TP=1, FP=0, FN=0
Image: SNAGDEEA-0115_png_jpg.rf.a4bb6fc2f52cc7e7fc3e026624497209
Class 34: TP=1, FP=0, FN=0
Class 39: TP=1, FP=0, FN=0
Image: SNAGDEEA-0217_png_jpg.rf.c6469ccd13cf62431342bcee79137691
Class 40: TP=1, FP=0, FN=0
Image: SNAG12-RADU_839_png.rf.a9b8a128cb9c7530d6c4128c95c5c6dd
Class 2: TP=1, FP=0, FN=0
Class 26: TP=1, FP=0, FN=0
Image: SNAGDEEA-0186_png_jpg.rf.8f909d7345b764978625444b36c2b203
Class 39: TP=1, FP=0, FN=0
Image: SNAG12-RADU_862_png.rf.e6c3c8c1903e94344d7e765ee1f5872b
Class 2: TP=1, FP=

In [ ]:
import os
import pandas as pd
from collections import defaultdict

def bbox_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calculate the coordinates of the intersection rectangle
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    # Calculate the area of intersection rectangle
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    # Calculate the area of both bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    # Calculate the IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    
    return iou

def calculate_metrics(gt_dir, pred_dir, iou_threshold=0.5):
    """
    Calculate performance metrics for object detection.
    """
    gt_boxes = defaultdict(lambda: defaultdict(list))  # {class_id: {image_name: [...]}}
    pred_boxes = defaultdict(lambda: defaultdict(list))  # {class_id: {image_name: [...]}}
    
    # Parse ground truth files
    for gt_file in os.listdir(gt_dir):
        image_name = os.path.splitext(gt_file)[0]
        with open(os.path.join(gt_dir, gt_file), 'r') as f:
            for line in f:
                class_id, x, y, w, h = map(float, line.split())
                gt_boxes[int(class_id)][image_name].append((x, y, w, h))
    
    # Parse predicted files
    for pred_file in os.listdir(pred_dir):
        image_name = os.path.splitext(pred_file)[0]
        with open(os.path.join(pred_dir, pred_file), 'r') as f:
            for line in f:
                parts = line.split()
                class_id, x, y, w, h = map(float, parts[:5])
                if len(parts) > 5:
                    conf = float(parts[5])
                else:
                    conf = 1.0  # Assign a default confidence score of 1.0
                pred_boxes[int(class_id)][image_name].append((x, y, w, h, conf))
    
    metrics = defaultdict(dict)
    
    for class_id in set(gt_boxes.keys()) | set(pred_boxes.keys()):
        tp, fp, fn = 0, 0, 0
        gt_count = sum(len(boxes) for boxes in gt_boxes[class_id].values())
        pred_count = sum(len(boxes) for boxes in pred_boxes[class_id].values())
        
        # Sort predicted boxes by confidence
        sorted_pred_boxes = []
        for image_name, boxes in pred_boxes[class_id].items():
            sorted_pred_boxes.extend(sorted(boxes, key=lambda x: x[-1], reverse=True))
        
        # Calculate TP, FP, and FN
        for pred_box in sorted_pred_boxes:
            found = False
            for image_name, gt_boxes in gt_boxes[class_id].items():
                for gt_box in gt_boxes:
                    iou = bbox_iou(pred_box[:4], gt_box)
                    if iou >= iou_threshold:
                        tp += 1
                        gt_boxes[class_id][image_name].remove(gt_box)
                        found = True
                        break
                if found:
                    break
            if not found:
                fp += 1
        fn = gt_count - tp
        
        # Calculate precision, recall, and F-score
        if tp + fp > 0:
            precision = tp / (tp + fp)
        else:
            precision = 0
        if tp + fn > 0:
            recall = tp / (tp + fn)
        else:
            recall = 0
        if precision + recall > 0:
            f_score = 2 * (precision * recall) / (precision + recall)
        else:
            f_score = 0
        
        # Store metrics
        metrics[class_id] = {
            'TP': tp,
            'FP': fp,
            'FN': fn,
            'pred_count': pred_count,
            'gt_count': gt_count,
            'precision': precision,
            'recall': recall,
            'f_score': f_score
        }
    
    # Calculate average precision
    for class_id in metrics.keys():
        ap = 0
        precisions = []
        recalls = []
        sorted_pred_boxes = []
        for image_name, boxes in pred_boxes[class_id].items():
            sorted_pred_boxes.extend(sorted(boxes, key=lambda x: x[-1], reverse=True))
        for conf in sorted([box[-1] for box in sorted_pred_boxes], reverse=True):
            tp, fp, fn = 0, 0, 0
            for pred_box in sorted_pred_boxes:
                if pred_box[-1] >= conf:
                    found = False
                    for image_name, gt_boxes in gt_boxes[class_id].items():
                        for gt_box in gt_boxes:
                            iou = bbox_iou(pred_box[:4], gt_box)
                            if iou >= iou_threshold:
                                tp += 1
                                gt_boxes[class_id][image_name].remove(gt_box)
                                found = True
                                break
                        if found:
                            break
                    if not found:
                        fp += 1
                else:
                    fn += sum(len(boxes) for image_name, boxes in gt_boxes[class_id].items())
                    break
            if tp + fp > 0:
                precision = tp / (tp + fp)
            else:
                precision = 0
            if tp + fn > 0:
                recall = tp / (tp + fn)
            else:
                recall = 0
            precisions.append(precision)
            recalls.append(recall)
            ap += precision * (recalls[-1] - recalls[-2]) if len(recalls) > 1 else precision * recalls[0]
        metrics[class_id]['ap'] = ap
    
    # Create a DataFrame with all metrics
    df = pd.DataFrame.from_dict(metrics, orient='index')
    
    return df

In [ ]:
gt_dir = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''
pred_dir = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
metrics_df = calculate_metrics(gt_dir, pred_dir)

IndexError: list index out of range

In [ ]:
prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics import precision_recall_curve, average_precision_score

def iou(box1, box2):
    """Calculate Intersection over Union (IoU) of two bounding boxes."""
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    if inter_x_min < inter_x_max and inter_y_min < inter_y_max:
        inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    else:
        inter_area = 0

    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area

def parse_data(file_list, is_prediction=False):
    data = defaultdict(list)
    for file in file_list:
        with open(file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                bbox = list(map(float, parts[1:5]))
                if is_prediction:
                    confidence = float(parts[5])
                    data[class_id].append(bbox + [confidence])
                else:
                    data[class_id].append(bbox)
    return data

def calculate_metrics(gt_data, pred_data, iou_threshold=0.5):
    metrics = []

    for class_id in sorted(set(gt_data.keys()).union(pred_data.keys())):
        gt_boxes = gt_data.get(class_id, [])
        pred_boxes = pred_data.get(class_id, [])

        gt_flags = np.zeros(len(gt_boxes))
        pred_flags = np.zeros(len(pred_boxes))

        # Sort predictions by confidence score
        pred_boxes = sorted(pred_boxes, key=lambda x: x[4], reverse=True)

        # Match predictions to ground truths
        for i, pred_box in enumerate(pred_boxes):
            pred_bbox = pred_box[:4]
            pred_conf = pred_box[4]
            best_iou = 0
            best_gt_idx = -1

            for j, gt_box in enumerate(gt_boxes):
                if gt_flags[j] == 0:
                    iou_score = iou(pred_bbox, gt_box)
                    if iou_score > best_iou:
                        best_iou = iou_score
                        best_gt_idx = j

            if best_iou >= iou_threshold:
                pred_flags[i] = 1
                gt_flags[best_gt_idx] = 1

        tp = np.sum(pred_flags)
        fp = len(pred_flags) - tp
        fn = len(gt_flags) - np.sum(gt_flags)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        fscore = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Average Precision calculation
        pred_scores = np.array([box[4] for box in pred_boxes])
        true_labels = pred_flags.astype(int)
        average_precision = average_precision_score(true_labels, pred_scores)

        metrics.append({
            'class_id': class_id,
            'precision': precision,
            'recall': recall,
            'fscore': fscore,
            'average_precision': average_precision,
            'TP': tp,
            'FP': fp,
            'FN': fn
        })

    return pd.DataFrame(metrics)

# Example usage

prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

gt_files = [os.path.join(ground_truth_folder, f) for f in os.listdir(ground_truth_folder) if f.endswith('.txt')]
pred_files = [os.path.join(prediction_folder, f) for f in os.listdir(prediction_folder) if f.endswith('.txt')]

gt_data = parse_data(gt_files)
pred_data = parse_data(pred_files, is_prediction=True)

metrics_df = calculate_metrics(gt_data, pred_data)
# print(metrics_df)


In [12]:
metrics_df

,class_id,precision,recall,fscore,average_precision,TP,FP,FN
0,1,0.060000,0.063830,0.061856,0.456439,3.0,47.0,44.0
1,2,0.000000,0.000000,0.000000,-0.000000,0.0,20.0,14.0
2,3,0.000000,0.000000,0.000000,-0.000000,0.0,6.0,4.0
3,4,0.000000,0.000000,0.000000,-0.000000,0.0,18.0,16.0
4,8,0.000000,0.000000,0.000000,-0.000000,0.0,11.0,12.0
5,9,0.000000,0.000000,0.000000,-0.000000,0.0,12.0,12.0
6,10,0.000000,0.000000,0.000000,-0.000000,0.0,1.0,0.0
7,11,0.000000,0.000000,0.000000,-0.000000,0.0,3.0,2.0
8,12,0.000000,0.000000,0.000000,-0.000000,0.0,2.0,1.0
9,13,0.000000,0.000000,0.000000,-0.000000,0.0,1.0,0.0


In [10]:
metrics_df

,class_id,precision,recall,fscore,average_precision,TP,FP,FN
0,1,0.065217,0.063830,0.064516,0.061183,3.0,43.0,44.0
1,2,0.000000,0.000000,0.000000,-0.000000,0.0,16.0,14.0
2,3,0.000000,0.000000,0.000000,-0.000000,0.0,7.0,4.0
3,4,0.000000,0.000000,0.000000,-0.000000,0.0,18.0,16.0
4,8,0.000000,0.000000,0.000000,-0.000000,0.0,10.0,12.0
5,9,0.000000,0.000000,0.000000,-0.000000,0.0,16.0,12.0
6,10,0.000000,0.000000,0.000000,-0.000000,0.0,3.0,0.0
7,11,0.000000,0.000000,0.000000,-0.000000,0.0,2.0,2.0
8,12,0.000000,0.000000,0.000000,-0.000000,0.0,4.0,1.0
9,14,0.000000,0.000000,0.000000,-0.000000,0.0,2.0,0.0


In [3]:
import os
import numpy as np
from collections import defaultdict
import pandas as pd

def load_labels(folder):
    labels = {}
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
                image_labels = []
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    image_labels.append(parts)
                labels[filename] = image_labels
    return labels

def iou(box1, box2):
    x1_min = box1[1] - box1[3] / 2
    x1_max = box1[1] + box1[3] / 2
    y1_min = box1[2] - box1[4] / 2
    y1_max = box1[2] + box1[4] / 2
    
    x2_min = box2[1] - box2[3] / 2
    x2_max = box2[1] + box2[3] / 2
    y2_min = box2[2] - box2[4] / 2
    y2_max = box2[2] + box2[4] / 2
    
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    intersection = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union = box1_area + box2_area - intersection
    return intersection / union

def compute_tp_fp_fn(pred_labels, gt_labels, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    matched_gt_indices = set()
    
    for pred_box in pred_labels:
        best_iou = 0
        best_gt_index = -1
        for gt_index, gt_box in enumerate(gt_labels):
            if gt_box[0] == pred_box[0]:  # same class
                current_iou = iou(pred_box, gt_box)
                if current_iou > best_iou:
                    best_iou = current_iou
                    best_gt_index = gt_index
        if best_iou > iou_threshold:
            if best_gt_index not in matched_gt_indices:
                tp += 1
                matched_gt_indices.add(best_gt_index)
            else:
                fp += 1
        else:
            fp += 1
    
    fn = len(gt_labels) - len(matched_gt_indices)
    
    return tp, fp, fn

def compute_precision_recall(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def compute_metrics(pred_labels, gt_labels, iou_threshold=0.5):
    class_stats = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0, 'curate_count': 0, 'pred_count': 0})
    
    for image_id in pred_labels:
        preds = pred_labels[image_id]
        gts = gt_labels.get(image_id, [])
        
        tp, fp, fn = compute_tp_fp_fn(preds, gts, iou_threshold)
        
        for pred in preds:
            class_stats[pred[0]]['pred_count'] += 1
        
        for gt in gts:
            class_stats[gt[0]]['curate_count'] += 1
            
        for class_id, stats in class_stats.items():
            stats['tp'] += tp
            stats['fp'] += fp
            stats['fn'] += fn
    
    results = []
    for class_id, stats in class_stats.items():
        tp, fp, fn = stats['tp'], stats['fp'], stats['fn']
        precision, recall = compute_precision_recall(tp, fp, fn)
        f1 = compute_f1(precision, recall)
        results.append({
            'class_label': int(class_id),
            'Curate count': stats['curate_count'],
            'Pred count': stats['pred_count'],
            'TP': tp,
            'FP': fp,
            'FN': fn,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })
    
    return pd.DataFrame(results)

In [6]:
# Define folders
prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V1_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

# Load labels
pred_labels = load_labels(prediction_folder)
gt_labels = load_labels(ground_truth_folder)

# Compute metrics
V1_class_stats_df = compute_metrics(pred_labels, gt_labels)

# Print DataFrame
V1_class_stats_df['class_label'] = V1_class_stats_df['class_label'].map(yolo_part_mapping)
# sort by class_label
V1_class_stats_df = V1_class_stats_df.sort_values(by='class_label')
V1_class_stats_df

,class_label,Curate count,Pred count,TP,FP,FN,Precision,Recall,F1 Score
3,forb_ahead,47,46,802,134,62,0.856838,0.928241,0.891111
11,forb_left,13,16,764,125,59,0.859393,0.928311,0.892523
21,forb_overtake,4,7,612,101,53,0.858345,0.920301,0.888244
12,forb_right,16,18,764,125,59,0.859393,0.928311,0.892523
30,forb_speed_over_20,12,10,513,84,40,0.859296,0.927667,0.892174
15,forb_speed_over_30,12,16,746,121,57,0.860438,0.929016,0.893413
18,forb_speed_over_40,0,3,709,112,55,0.863581,0.928010,0.894637
25,forb_speed_over_5,2,2,579,94,48,0.860327,0.923445,0.890769
16,forb_speed_over_50,1,4,745,119,56,0.862269,0.930087,0.894895
31,forb_speed_over_70,0,2,463,79,35,0.854244,0.929719,0.890385


In [7]:
# Define folders
prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

# Load labels
pred_labels = load_labels(prediction_folder)
gt_labels = load_labels(ground_truth_folder)

# Compute metrics
V2_class_stats_df = compute_metrics(pred_labels, gt_labels)

# Print DataFrame
V2_class_stats_df['class_label'] = V2_class_stats_df['class_label'].map(yolo_part_mapping)
# sort by class_label
V2_class_stats_df = V2_class_stats_df.sort_values(by='class_label')
V2_class_stats_df

,class_label,Curate count,Pred count,TP,FP,FN,Precision,Recall,F1 Score
3,forb_ahead,47,50,813,119,54,0.872318,0.937716,0.903835
11,forb_left,14,20,774,107,52,0.878547,0.937046,0.906854
19,forb_overtake,4,6,676,90,49,0.882507,0.932414,0.906774
12,forb_right,16,18,774,107,52,0.878547,0.937046,0.906854
31,forb_speed_over_20,12,11,518,66,35,0.886986,0.936709,0.911170
29,forb_speed_over_30,12,12,560,75,41,0.881890,0.931780,0.906149
32,forb_speed_over_40,0,1,502,64,32,0.886926,0.940075,0.912727
20,forb_speed_over_5,2,3,644,89,49,0.878581,0.929293,0.903226
18,forb_speed_over_50,1,2,684,90,49,0.883721,0.933151,0.907764
16,forb_speed_over_60,0,1,730,95,49,0.884848,0.937099,0.910224


In [5]:
import os
import pandas as pd
from collections import defaultdict

def bbox_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calculate the coordinates of the intersection rectangle
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    # Calculate the area of intersection rectangle
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    # Calculate the area of both bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    # Calculate the IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    
    return iou

def calculate_metrics(gt_dir, pred_dir, iou_threshold=0.5):
    """
    Calculate performance metrics for object detection.
    """
    gt_boxes = defaultdict(lambda: defaultdict(list))  # {class_id: {image_name: [...]}}
    pred_boxes = defaultdict(lambda: defaultdict(list))  # {class_id: {image_name: [...]}}
    
    # Parse ground truth files
    for gt_file in os.listdir(gt_dir):
        image_name = os.path.splitext(gt_file)[0]
        with open(os.path.join(gt_dir, gt_file), 'r') as f:
            for line in f:
                class_id, x, y, w, h = map(float, line.split())
                gt_boxes[int(class_id)][image_name].append((x, y, w, h))
    
    # Parse predicted files
    for pred_file in os.listdir(pred_dir):
        image_name = os.path.splitext(pred_file)[0]
        with open(os.path.join(pred_dir, pred_file), 'r') as f:
            for line in f:
                parts = line.split()
                class_id, x, y, w, h = map(float, parts[:5])
                if len(parts) > 5:
                    conf = float(parts[5])
                else:
                    conf = 1.0  # Assign a default confidence score of 1.0
                pred_boxes[int(class_id)][image_name].append((x, y, w, h, conf))
    
    metrics = defaultdict(dict)
    
    for class_id in set(gt_boxes.keys()) | set(pred_boxes.keys()):
        tp, fp, fn = 0, 0, 0
        gt_count = sum(len(boxes) for boxes in gt_boxes[class_id].values())
        pred_count = sum(len(boxes) for boxes in pred_boxes[class_id].values())
        
        # Sort predicted boxes by confidence
        sorted_pred_boxes = []
        for image_name, boxes in pred_boxes[class_id].items():
            sorted_pred_boxes.extend(sorted(boxes, key=lambda x: x[-1], reverse=True))
        
        # Calculate TP, FP, and FN
        for pred_box in sorted_pred_boxes:
            found = False
            for image_name, gt_boxes in gt_boxes[class_id].items():
                for gt_box in gt_boxes:
                    iou = bbox_iou(pred_box[:4], gt_box)
                    if iou >= iou_threshold:
                        tp += 1
                        gt_boxes[class_id][image_name].remove(gt_box)
                        found = True
                        break
                if found:
                    break
            if not found:
                fp += 1
        fn = gt_count - tp
        
        # Calculate precision, recall, and F-score
        if tp + fp > 0:
            precision = tp / (tp + fp)
        else:
            precision = 0
        if tp + fn > 0:
            recall = tp / (tp + fn)
        else:
            recall = 0
        if precision + recall > 0:
            f_score = 2 * (precision * recall) / (precision + recall)
        else:
            f_score = 0
        
        # Store metrics
        metrics[class_id] = {
            'TP': tp,
            'FP': fp,
            'FN': fn,
            'pred_count': pred_count,
            'gt_count': gt_count,
            'precision': precision,
            'recall': recall,
            'f_score': f_score
        }
    
    # Calculate average precision
    for class_id in metrics.keys():
        ap = 0
        precisions = []
        recalls = []
        sorted_pred_boxes = []
        for image_name, boxes in pred_boxes[class_id].items():
            sorted_pred_boxes.extend(sorted(boxes, key=lambda x: x[-1], reverse=True))
        for conf in sorted([box[-1] for box in sorted_pred_boxes], reverse=True):
            tp, fp, fn = 0, 0, 0
            for pred_box in sorted_pred_boxes:
                if pred_box[-1] >= conf:
                    found = False
                    for image_name, gt_boxes in gt_boxes[class_id].items():
                        for gt_box in gt_boxes:
                            iou = bbox_iou(pred_box[:4], gt_box)
                            if iou >= iou_threshold:
                                tp += 1
                                gt_boxes[class_id][image_name].remove(gt_box)
                                found = True
                                break
                        if found:
                            break
                    if not found:
                        fp += 1
                else:
                    fn += sum(len(boxes) for image_name, boxes in gt_boxes[class_id].items())
                    break
            if tp + fp > 0:
                precision = tp / (tp + fp)
            else:
                precision = 0
            if tp + fn > 0:
                recall = tp / (tp + fn)
            else:
                recall = 0
            precisions.append(precision)
            recalls.append(recall)
            ap += precision * (recalls[-1] - recalls[-2]) if len(recalls) > 1 else precision * recalls[0]
        metrics[class_id]['ap'] = ap
    
    # Create a DataFrame with all metrics
    df = pd.DataFrame.from_dict(metrics, orient='index')
    
    return df

In [6]:
gt_dir = r'D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'
pred_dir = r'D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'
metrics_df = calculate_metrics(gt_dir, pred_dir)
metrics_df

IndexError: list index out of range

# Get class wise accuracies

In [9]:
import os
import glob
from collections import defaultdict

# Specify the directories containing the ground truth and prediction files
gt_dir = r'D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'
pred_dir = r'D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'

def read_boxes(file_path):
    boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            values = line.strip().split()
            class_id = int(values[0])
            box = [float(x) for x in values[1:]]
            boxes.append((class_id, *box))
    return boxes

def read_pred_boxes(file_path):
    pred_boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            values = line.strip().split()
            class_id = int(values[0])
            box = [float(x) for x in values[1:5]]
            conf_score = float(values[5])
            pred_boxes.append((class_id, *box, conf_score))
    return pred_boxes

def compute_iou(box1, box2):
    """
    Compute the Intersection over Union (IoU) between two bounding boxes.
    """
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2

    # Calculate the coordinates of the intersection rectangle
    x_left = max(x1, x3)
    y_top = max(y1, y3)
    x_right = min(x2, x4)
    y_bottom = min(y2, y4)

    # Calculate the area of intersection rectangle
    intersection_area = max(0, x_right - x_left + 1) * max(0, y_bottom - y_top + 1)

    # Calculate the area of both bounding boxes
    box1_area = (x2 - x1 + 1) * (y2 - y1 + 1)
    box2_area = (x4 - x3 + 1) * (y4 - y3 + 1)

    # Compute the Intersection over Union
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou

def calculate_metrics(ground_truth, predictions, iou_threshold=0.5):
    # Initialize dictionaries to store true positives, false positives, and false negatives
    true_positives = defaultdict(int)
    false_positives = defaultdict(int)
    false_negatives = defaultdict(int)

    # Initialize dictionaries to store prediction counts and ground truth counts
    pred_counts = defaultdict(int)
    gt_counts = defaultdict(int)

    # Initialize a dictionary to store images with true positives for each class
    tp_images = defaultdict(list)

    for image_name, gt_boxes in ground_truth.items():
        if image_name in predictions:
            pred_boxes = predictions[image_name]
        else:
            pred_boxes = []

        # Count the ground truth boxes for each class
        for gt_box in gt_boxes:
            class_id = gt_box[0]
            gt_counts[class_id] += 1

        # Match predictions with ground truth
        for pred_box in pred_boxes:
            pred_class_id = pred_box[0]
            pred_counts[pred_class_id] += 1
            best_iou = 0
            best_gt_box = None

            # Find the best matching ground truth box
            for gt_box in gt_boxes:
                gt_class_id = gt_box[0]
                iou = compute_iou(pred_box[1:5], gt_box[1:])
                if iou > best_iou and gt_class_id == pred_class_id:
                    best_iou = iou
                    best_gt_box = gt_box

            # If the best IoU is above the threshold, consider it a true positive
            if best_iou > iou_threshold:
                true_positives[pred_class_id] += 1
                gt_boxes.remove(best_gt_box)
                tp_images[pred_class_id].append(image_name)
            else:
                false_positives[pred_class_id] += 1

        # Any remaining ground truth boxes are false negatives
        for gt_box in gt_boxes:
            gt_class_id = gt_box[0]
            false_negatives[gt_class_id] += 1

    # Calculate precision, recall, and F1-score for each class
    metrics = []
    for class_id in set(list(gt_counts.keys()) + list(pred_counts.keys())):
        tp = true_positives[class_id]
        fp = false_positives[class_id]
        fn = false_negatives[class_id]

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

        metrics.append({
            'class_id': class_id,
            'true_positives': tp,
            'false_positives': fp,
            'false_negatives': fn,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'pred_count': pred_counts[class_id],
            'gt_count': gt_counts[class_id]
        })

    # Calculate the average precision
    average_precision = np.mean([metric['precision'] for metric in metrics])

    return metrics, average_precision, tp_images

# Read the ground truth files
ground_truth = {}
for gt_file in glob.glob(os.path.join(gt_dir, '*.txt')):
    image_name = os.path.basename(gt_file)
    ground_truth[image_name] = read_boxes(gt_file)

# Read the prediction files
predictions = {}
for pred_file in glob.glob(os.path.join(pred_dir, '*.txt')):
    image_name = os.path.basename(pred_file)
    predictions[image_name] = read_pred_boxes(pred_file)

# Call the calculate_metrics function with the ground truth and predictions
metrics, average_precision, tp_images = calculate_metrics(ground_truth, predictions)

# # Print the results
# print("Performance Metrics:")
# for metric in metrics:
#     print(f"Class ID: {metric['class_id']}")
#     print(f"True Positives: {metric['true_positives']}")
#     print(f"False Positives: {metric['false_positives']}")
#     print(f"False Negatives: {metric['false_negatives']}")
#     print(f"Precision: {metric['precision']}")
#     print(f"Recall: {metric['recall']}")
#     print(f"F1-score: {metric['f1_score']}")
#     print(f"Prediction Count: {metric['pred_count']}")
#     print(f"Ground Truth Count: {metric['gt_count']}")
#     print()

# print(f"Average Precision: {average_precision}")

# print("Images with True Positives:")
# for class_id, image_names in tp_images.items():
#     print(f"Class ID: {class_id}")
#     print(f"Image Names: {', '.join(image_names)}")
#     print()

In [10]:
model_accuracy = pd.DataFrame(metrics)
model_accuracy['class_label'] = model_accuracy['class_id'].map(yolo_part_mapping)
model_accuracy

,class_id,true_positives,false_positives,false_negatives,precision,recall,f1_score,pred_count,gt_count,class_label
0,1,44,6,3,0.880000,0.936170,0.907216,50,47,forb_ahead
1,2,14,6,0,0.700000,1.000000,0.823529,20,14,forb_left
2,3,4,2,0,0.666667,1.000000,0.800000,6,4,forb_overtake
3,4,16,2,0,0.888889,1.000000,0.941176,18,16,forb_right
4,8,10,1,2,0.909091,0.833333,0.869565,11,12,forb_speed_over_20
5,9,11,1,1,0.916667,0.916667,0.916667,12,12,forb_speed_over_30
6,10,0,1,0,0.000000,0.000000,0.000000,1,0,forb_speed_over_40
7,11,2,1,0,0.666667,1.000000,0.800000,3,2,forb_speed_over_5
8,12,1,1,0,0.500000,1.000000,0.666667,2,1,forb_speed_over_50
9,13,0,1,0,0.000000,0.000000,0.000000,1,0,forb_speed_over_60


In [20]:
fp_images[9]

['SNAGDEEA-0211_png_jpg.rf.0441753301318b14978a1f4d492f9b0a.txt']

In [ ]:
tp_images, fp_images, fn_images

# Confusion Quadrant

In [ ]:
import os
import glob
from collections import defaultdict

# Specify the directories containing the ground truth and prediction files
gt_dir = r'D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'
pred_dir = r'D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'

def read_boxes(file_path):
    boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            values = line.strip().split()
            class_id = int(values[0])
            box = [float(x) for x in values[1:]]
            boxes.append((class_id, *box))
    return boxes

def read_pred_boxes(file_path):
    pred_boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            values = line.strip().split()
            class_id = int(values[0])
            box = [float(x) for x in values[1:5]]
            conf_score = float(values[5])
            pred_boxes.append((class_id, *box, conf_score))
    return pred_boxes

def compute_iou(box1, box2):
    """
    Compute the Intersection over Union (IoU) between two bounding boxes.
    """
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2

    # Calculate the coordinates of the intersection rectangle
    x_left = max(x1, x3)
    y_top = max(y1, y3)
    x_right = min(x2, x4)
    y_bottom = min(y2, y4)

    # Calculate the area of intersection rectangle
    intersection_area = max(0, x_right - x_left + 1) * max(0, y_bottom - y_top + 1)

    # Calculate the area of both bounding boxes
    box1_area = (x2 - x1 + 1) * (y2 - y1 + 1)
    box2_area = (x4 - x3 + 1) * (y4 - y3 + 1)

    # Compute the Intersection over Union
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou

def calculate_metrics(ground_truth, predictions, iou_threshold=0.5):
    # Initialize dictionaries to store true positives, false positives, and false negatives
    true_positives = defaultdict(int)
    false_positives = defaultdict(int)
    false_negatives = defaultdict(int)

    # Initialize dictionaries to store prediction counts and ground truth counts
    pred_counts = defaultdict(int)
    gt_counts = defaultdict(int)

    # Initialize dictionaries to store images with true positives, false positives, and false negatives for each class
    tp_images = defaultdict(list)
    fp_images = defaultdict(list)
    fn_images = defaultdict(list)

    for image_name, gt_boxes in ground_truth.items():
        if image_name in predictions:
            pred_boxes = predictions[image_name]
        else:
            pred_boxes = []

        # Count the ground truth boxes for each class
        for gt_box in gt_boxes:
            class_id = gt_box[0]
            gt_counts[class_id] += 1

        # Match predictions with ground truth
        for pred_box in pred_boxes:
            pred_class_id = pred_box[0]
            pred_counts[pred_class_id] += 1
            best_iou = 0
            best_gt_box = None

            # Find the best matching ground truth box
            for gt_box in gt_boxes:
                gt_class_id = gt_box[0]
                iou = compute_iou(pred_box[1:5], gt_box[1:])
                if iou > best_iou and gt_class_id == pred_class_id:
                    best_iou = iou
                    best_gt_box = gt_box

            # If the best IoU is above the threshold, consider it a true positive
            if best_iou > iou_threshold:
                true_positives[pred_class_id] += 1
                gt_boxes.remove(best_gt_box)
                tp_images[pred_class_id].append(image_name)
            else:
                false_positives[pred_class_id] += 1
                fp_images[pred_class_id].append(image_name)

        # Any remaining ground truth boxes are false negatives
        for gt_box in gt_boxes:
            gt_class_id = gt_box[0]
            false_negatives[gt_class_id] += 1
            fn_images[gt_class_id].append(image_name)

    # Calculate precision, recall, and F1-score for each class
    metrics = []
    for class_id in set(list(gt_counts.keys()) + list(pred_counts.keys())):
        tp = true_positives[class_id]
        fp = false_positives[class_id]
        fn = false_negatives[class_id]

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

        metrics.append({
            'class_id': class_id,
            'true_positives': tp,
            'false_positives': fp,
            'false_negatives': fn,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'pred_count': pred_counts[class_id],
            'gt_count': gt_counts[class_id]
        })

    # Calculate the average precision
    average_precision = np.mean([metric['precision'] for metric in metrics])

    return metrics, average_precision, tp_images, fp_images, fn_images

# Read the ground truth files
ground_truth = {}
for gt_file in glob.glob(os.path.join(gt_dir, '*.txt')):
    image_name = os.path.basename(gt_file)
    ground_truth[image_name] = read_boxes(gt_file)

# Read the prediction files
predictions = {}
for pred_file in glob.glob(os.path.join(pred_dir, '*.txt')):
    image_name = os.path.basename(pred_file)
    predictions[image_name] = read_pred_boxes(pred_file)

# Call the calculate_metrics function with the ground truth and predictions
metrics, average_precision, tp_images, fp_images, fn_images = calculate_metrics(ground_truth, predictions)

# # Print the results
# print("Performance Metrics:")
# for metric in metrics:
#     print(f"Class ID: {metric['class_id']}")
#     print(f"True Positives: {metric['true_positives']}")
#     print(f"False Positives: {metric['false_positives']}")
#     print(f"False Negatives: {metric['false_negatives']}")
#     print(f"Precision: {metric['precision']}")
#     print(f"Recall: {metric['recall']}")
#     print(f"F1-score: {metric['f1_score']}")
#     print(f"Prediction Count: {metric['pred_count']}")
#     print(f"Ground Truth Count: {metric['gt_count']}")
#     print()

# print(f"Average Precision: {average_precision}")

# print("True Positive Images:")
# for class_id, image_names in tp_images.items():
#     print(f"Class ID: {class_id}")
#     print(f"Image Names: {', '.join(image_names)}")
#     print()

# print("False Positive Images")

In [ ]:
tp_images

ValueError: All arrays must be of the same length